In [19]:
import time
import os
import requests
import json
from datetime import datetime
import pandas as pd

def json_html(parameters):
    response = requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json", params=parameters)
    data = json.loads(response.text)
    content = data['response']['docs']
    return data, content
    
def html_txt(data, parameters, lst, path, size=10):
    for i in range(size):
        try:
            url = data['response']['docs'][i]['web_url']
            response = requests.get(url)
            
        except :
            continue
        else:
            file = url.split('https://')[1]
            file1 = file.replace('/',':')
            lst.append(file1)
            filename = path+file1
            f = open(filename,'w') 
            f.write(response.text)
            f.close()
            
    return lst
    
def NYT_API(query, path, start_date, end_date):
    lst=[]
    array=[]
    parameters = {}
    parameters['api-key']="JwLKzlcdwaGqOXqeiGzZGSiOmO8WN2Oi"
    parameters['q']=query
    parameters['begin_date']=''
    parameters['end_date']=''
    parameters['sort']='oldest'
    
    parameters['begin_date'] = start_date
    parameters['end_date'] = end_date
    data, content = json_html(parameters)
    ttl_message = data['response']['meta']['hits']
    print(ttl_message)

    for i in range(round(ttl_message/10)):
        parameters['page']=i
        data, content = json_html(parameters)
        lst=html_txt(data, parameters, lst, path)
        array.append(content)
        if i%9==0:
            time.sleep(80)

    parameters['page']=round(ttl_message/10)
    data, content = json_html(parameters)
    array.append(content)
    lst=html_txt(data, parameters, lst, path, ttl_message%10)
    filename = path+'index.json'
    with open(filename,'w') as outfile:
        json.dump(array, outfile, sort_keys=True, indent=4, separators=(',',':'))
    print(lst)
    
query =['Apple Inc.', 'Alibaba', 'Amazon', 'Google', 'Facebook', 'Netflix', 
                'Microsoft', 'Tesla', 'eBay', 'Cisco', 'economy','finance','stocks','stocks market']

date_Dict = {}
df = pd.date_range(start='1/1/2016', periods=12, freq='M')
for date in df:
    end_date = datetime.strftime(date, '%Y-%m-%d')
    date = end_date.split('-')[:-1]
    date.append('01')
    start_date = '-'.join(a for a in date)
    date_Dict[start_date] = end_date
    
for key, value in date_Dict.items():
    for a in query:
        year = key[:4]
        month = key[4:6]
        path ='../../Data/NYT/{}/{}/{}/'.format(year,month, a)
        os.makedirs(path)
        print(key)
        print(value)
        NYT_API(a, path, key, value)
        time.sleep(65)